#### Spotlight

The Quickstart Example

In [1]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

In [2]:
dataset = get_movielens_dataset(variant='100K')

train, test = random_train_test_split(dataset)

In [3]:
train

<Interactions dataset (944 users x 1683 items x 80000 interactions)>

Not helpful... need to go deeper:

In [4]:
train.user_ids

array([ 95, 585, 332, ..., 377,  35, 474], dtype=int32)

In [5]:
train.item_ids

array([ 78, 652, 105, ..., 271, 748, 463], dtype=int32)

In [6]:
train.ratings

array([3., 4., 2., ..., 4., 4., 5.], dtype=float32)

And peek inside:

In [7]:
train.tocsr().todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 5., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 5., ..., 0., 0., 0.]], dtype=float32)

The model:

In [8]:
model = ImplicitFactorizationModel(n_iter=3,
                                   loss='bpr')
model.fit(train)

mrr = mrr_score(model, test)

In [9]:
mrr[:10]

array([0.01228298, 0.03906301, 0.02762595, 0.04638586, 0.04801733,
       0.01561031, 0.01174183, 0.14971021, 0.00413361, 0.0508525 ])

### GitHub Stars

Data retrived from scraping GitHub:

In [10]:
import pandas as pd 

df = pd.read_csv('data/stars.csv')

In [11]:
df.sample(5)

,user,repo,description,language,stargazers
62912,jcmkk3,lukejacksonn/ijk,Transforms arrays into virtual dom trees; a te...,JavaScript,428
29221,tlentali,NVIDIA/DALI,A library containing both highly optimized bui...,C++,2404
37801,jocobtt,sassoftware/python-swat,The SAS Scripting Wrapper for Analytics Transf...,Python,86
46195,tanelpuhu,sqshq/sampler,"Tool for shell commands execution, visualizati...",Go,7775
54295,tones7778,NabuCasa/hass-nabucasa,Issues related to the cloud integration in Nab...,Python,31


In [12]:
df['language'].value_counts()

Python              25702
JavaScript           6459
Jupyter Notebook     3777
Go                   2571
C++                  2544
                    ...  
CLIPS                   1
ASP                     1
Ren'Py                  1
API Blueprint           1
VCL                     1
Name: language, Length: 178, dtype: int64

In [13]:
df = df[df.language == 'Python']
df = df[~df['repo'].isin(['maxhumber/gif', 'maxhumber/gazpacho'])]

In [14]:
df.shape

(25380, 5)

In [15]:
len(df['repo'].unique())

12222

In [16]:
len(df['user'].unique())

326

In [17]:
df.head(3)

,user,repo,description,language,stargazers
0,sbarman-mi9,as-ideas/ForwardTacotron,⏩ Generating speech in a single forward pass w...,Python,97
1,sbarman-mi9,abhishekkrthakur/bert-sentiment,NaN,Python,21
4,sbarman-mi9,EmilyAlsentzer/clinicalBERT,repository for Publicly Available Clinical BER...,Python,160


In [18]:
from spotlight.interactions import Interactions

In [19]:
# won't work
interactions = Interactions(df['user'], df['repo'])

TypeError: must be str, not int

"Everything must be a number"

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

In [22]:
users = user_encoder.fit_transform(df['user'])
items = item_encoder.fit_transform(df['repo'])

In [23]:
interactions = Interactions(users, items)

In [24]:
interactions

<Interactions dataset (326 users x 12222 items x 25380 interactions)>

Be a good Data Scientist:

In [25]:
import numpy as np
from spotlight.cross_validation import random_train_test_split

train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=np.random.RandomState(42))

In [26]:
model = ImplicitFactorizationModel(loss='pointwise', n_iter=20)

In [27]:
model.fit(train)

In [28]:
df.head(3)

,user,repo,description,language,stargazers
0,sbarman-mi9,as-ideas/ForwardTacotron,⏩ Generating speech in a single forward pass w...,Python,97
1,sbarman-mi9,abhishekkrthakur/bert-sentiment,NaN,Python,21
4,sbarman-mi9,EmilyAlsentzer/clinicalBERT,repository for Publicly Available Clinical BER...,Python,160


Examining one user:

In [29]:
garrry = df[df['user'] == 'garrrychan']

In [30]:
garrry['repo'].values

array(['mnielsen/neural-networks-and-deep-learning',
       'brendan-rius/jupyter-c-kernel', 'google-research/uda',
       'hhatto/autopep8', 'MaxHalford/prince',
       'scikit-learn-contrib/sklearn-pandas', 'modin-project/modin',
       'VikParuchuri/apartment-finder', 'uber-research/parallax',
       'prabhupant/python-ds', 'tensorflow/nmt',
       'huggingface/transformers', 'h5py/h5py', 'google-research/bert',
       'sloria/TextBlob', 'openai/gpt-2', 'matsui528/rii',
       'ResidentMario/missingno', 'lmcinnes/enstop',
       'phatpiglet/autocorrect', 'barrust/pyspellchecker',
       'seatgeek/fuzzywuzzy', 'maxhumber/chart', 'apache/airflow',
       'graphql-python/graphene', 'maxhumber/marc', 'spotify/luigi',
       'garrrychan/recipe_recommender_system',
       'CamDavidsonPilon/lifetimes', 'maciejkula/spotlight',
       'lyst/lightfm', 'practical-recommender-systems/moviegeek',
       'uwescience/TrafficCruising-DSSG2017'], dtype=object)

Looking at the user_id:

In [31]:
u = user_encoder.transform(['garrrychan'])

And all of the items:

In [32]:
item_encoder.classes_

array(['00111000/Imports-in-Python', '05bit/peewee-async',
       '0Kee-Team/WatchAD', ..., 'zzw922cn/Automatic_Speech_Recognition',
       'zzzDavid/ICDAR-2019-SROIE', 'zzzeek/sqlalchemy'], dtype=object)

In [33]:
preds = model.predict(u, np.arange(len(item_encoder.classes_)))

In [34]:
pd.DataFrame({
    'repo': item_encoder.classes_,
    'pred': preds
}).sort_values('pred', ascending=False)

,repo,pred
11599,vinta/awesome-python,15.192618
9238,psf/black,12.725516
4836,encode/starlette,11.802556
9446,python/cpython,10.534921
5104,faif/python-patterns,9.884838
...,...,...
1066,Kr1s77/awesome-python-login-model,-13.754693
1579,PyCQA/redbaron,-13.789619
2738,anishathalye/dotbot,-13.803130
10256,sedthh/pyxelate,-14.246572


Evaulating the model:

In [35]:
from spotlight.evaluation import precision_recall_score

In [36]:
precision, recall = precision_recall_score(model, test, train, k=10)

In [37]:
precision.mean()

0.03104575163398693

In [38]:
recall.mean()

0.022731866150943074

Serialize:

In [39]:
import torch 

torch.save(model, 'model.spot')

In [40]:
del model

In [41]:
model = torch.load('model.spot')

Predict on another random user:

In [42]:
u = user_encoder.transform(['RandomOS'])

In [43]:
model.predict(u, np.arange(len(item_encoder.classes_)))

array([-8.196298 , -1.6531161, -6.206962 , ..., -9.286883 , -5.755635 ,
       -6.2078385], dtype=float32)

In [44]:
pd.DataFrame({
    'repo': item_encoder.classes_,
    'pred': model.predict(u, np.arange(len(item_encoder.classes_)))
}).sort_values('pred', ascending=False).head(20)

,repo,pred
4833,encode/httpx,12.698281
9090,pipxproject/pipx,11.746658
1256,Miserlou/Zappa,10.961097
4813,emeryberger/scalene,10.927866
9446,python/cpython,10.679359
1626,Qix-/better-exceptions,9.908401
11142,tiangolo/fastapi,8.883745
11292,tqdm/tqdm,8.881335
10087,samuelcolvin/pydantic,8.748564
9238,psf/black,8.681081


Actual likes:

In [45]:
df[df['user'] == 'RandomOS']['repo']

58885               tortoise/tortoise-orm
58887                 mingrammer/diagrams
58905                       spulec/uncurl
58913         mozilla-iot/webthing-python
58922                       spotify/luigi
58924                    python-trio/trio
58925                       ranger/ranger
58928              googlefonts/noto-emoji
58937                Synss/python-mbedtls
58940                    pipxproject/pipx
58941                       linkedin/shiv
58942                      pantsbuild/pex
58946        PythonCharmers/python-future
58947     alan-turing-institute/CleverCSV
58948                 emeryberger/scalene
58949                 aouinizied/nfstream
58950               ionelmc/python-hunter
58951                      wolever/pip2pi
58956          fossasia/open-event-server
58994                python-poetry/poetry
58997                     thumbor/thumbor
59004                       holoviz/panel
59019             0xInfection/Awesome-WAF
59020    swisskyrepo/PayloadsAllTh

### But what if the user is brand new?~

In [46]:
user_encoder.transform(['maxhumber'])

ValueError: y contains previously unseen labels: ['maxhumber']

In [47]:
model.predict(1993, np.arange(len(item_encoder.classes_)))

ValueError: Maximum user id greater than number of users in model.